<div style="text-align: center; font-size: 40px;">
    <b>Final Project</b>
    <br>
    Jarry Guillaume
    <br>
    
</div>


In [28]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import arff
import pandas as pd
import kmedoids 
import os 
import glob
from sklearn.svm import OneClassSVM
from hmmlearn.hmm import CategoricalHMM
from sklearn.svm import SVC
from tqdm import tqdm
import random
from BenchMark.markov_dataset_generator import MarkovDatasetGenerator
from BenchMark.markov_sequence import MarkovSequenceGenerator

For this project, we will use the categorical Datasets from the ADRepository-Anomaly-detection-datasets github repository. It is available here : 

- https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets?tab=readme-ov-fil

Since our article is focused on aonmaly detection for discrete timeseries, these dataset will allow us to deploy some of the techniques showcased in the article. Let's start ! 

In [333]:
folder_path = "ADRepository-Anomaly-detection-datasets/categorical data/"
datasets = []

for filepath in glob.glob(os.path.join(folder_path, "*")):
    try: 
        data, meta = arff.loadarff(filepath)
        datasets.append((data, meta))
    except: 
        print(f"Error while parsing file : {filepath}")

## Intro : Generating Synthetic Data :

Since the data we tried to find online for synthetic timeseries was rarely labeled, we propose to generate some synthetic data so that we can test and implement as many algorithm for our discreet anomaly detection library. We can then test our algorithm on some real, less labeled data.

### Markovian models :     

This class will generate synthetic data that creates Markovian Discreet sequences.

In [2]:
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

And now let us generate a dataset mixing a hidden Markow model and two markow models and let us wrap them up into the same dataset. We will also add an anomaly dataset, which will be another Markov model, with different probabilities.

In [3]:
transition_matrix1 = np.array([
    [0.7, 0.2, 0.1],
    [0.1, 0.7, 0.2],
    [0.2, 0.1, 0.7]
])

hidden_matrix1 = np.array([
    [0.7, 0.2, 0.1],
    [0.1, 0.8, 0.1],
    [0.2, 0.2, 0.6]
])

transition_matrix2 = np.array([
    [0.6, 0.3, 0.1],  # Emission distribution from hidden state 0
    [0.2, 0.5, 0.3],  # Emission distribution from hidden state 1
    [0.1, 0.2, 0.7]   # Emission distribution from hidden state 2
])

N = 5  
transition_matrix3 = np.random.rand(N, N)  
row_sums = transition_matrix3.sum(axis=1, keepdims=True)
transition_matrix3 = transition_matrix3 / row_sums

transition_matrices = [transition_matrix1, transition_matrix2, transition_matrix3]
hidden_matrices = [hidden_matrix1, None, None ]

In [ ]:
generator = MarkovDatasetGenerator(transition_matrices, hidden_matrices, n_sequences=200, sequence_length=50)
train_dataset = generator.generate()

In [5]:
N = 3
transition_matrix4 = np.random.rand(N, N)  
row_sums = transition_matrix4.sum(axis=1, keepdims=True)
transition_matrix4 = transition_matrix4 / row_sums
test_generator = MarkovDatasetGenerator([transition_matrix4, transition_matrix3], [None, None], n_sequences=20, sequence_length=50)
test_dataset = test_generator.generate()

# I) Kernell-Based Techniques 

In [ ]:
class KernellBase: 
    def __init__(self, dataset, similarity_metric): 
        self.dataset = dataset
        self.similarity_metric = similarity_metric
        self.similarity_matrix = None
        self.medoids = None

    def compute_similarity_matrix(self): 
        n = len(self.dataset)
        self.similarity_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(i, n):
                sim = self.similarity_metric(self.dataset[i], self.dataset[j])
                self.similarity_matrix[i,j] = sim
                self.similarity_matrix[j,i] = sim  # Symmetric
        
        self.distance_matrix = 1 - self.similarity_matrix
        return self.similarity_matrix
    
    def compute_kemedoids(self, kmax=10, kmin=1): 
        km = kmedoids.dynmsc(self.distance_matrix, kmax, kmin)
        self.medoids = [self.dataset[medoid] for medoid in km.medoids]
        return self.medoids

    def knearest_predict(self, test_sequence, k_nearest=5):
        similarities = []
        for sequence in self.dataset:  
            similarities.append(self.similarity_metric(test_sequence, sequence))
        
        similarities.sort(reverse=True)
        anomaly_score = 1 / similarities[k_nearest]
        return anomaly_score

    def clustering_predict(self, test_sequence, kmax=10, kmin=1):
        if self.similarity_matrix is None: 
            self.compute_similarity_matrix()
        if self.medoids is None:
            self.compute_kemedoids(kmax=kmax, kmin=kmin)

        max_similarity = 0
        for medoid in self.medoids: 
            max_similarity = max(max_similarity, self.similarity_metric(test_sequence, medoid))
        
        return 1 / max_similarity       

Now let us try our Kernell based methods with the longest common sequence kernell suggested in the article.

In [ ]:
def LCS_length(seq1, seq2):
    len1, len2 = len(seq1), len(seq2)
    dp = [[0]*(len2+1) for _ in range(len1+1)]
    for i in range(1, len1+1):
        for j in range(1, len2+1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1] + 1
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])
    return dp[len1][len2]

def nLCS(seq1, seq2):
    lcs = LCS_length(seq1, seq2)
    return lcs / ( (len(seq1)*len(seq2))**0.5 )

In [ ]:
kernell_based = KernellBase(train_dataset, nLCS)
distance_matrix = kernell_based.compute_similarity_matrix()
medoids = kernell_based.compute_kemedoids()

# II) Window Based Techniques :

## Window Based Structure

In [ ]:
class WindowBasedStruct: 
    def __init__(self, dataset, window_length, mode="average"): 
        self.window_length = window_length
        self.dataset = self.partition(dataset)
        self.mode = self.mode

    def partition(self, dataset): 
        partition = []
        for sequence in dataset: 
            partition = [sequence[i*self.window_length:(i+1)*self.window_length] for i in range(int(len(sequence) // self.window_length))]
            partition.append(partition)
        
        return partition

    def process_anomaly(self, anomaly_scores): 
        if self.mode == "average": 
            return np.mean(anomaly_scores)
        elif self.mode == "max": 
            return np.max(anomaly_scores)

## Lookahead Method

In [ ]:

class Lookahead(WindowBasedStruct): 
    def __init__(self, dataset, window_length, mode="average", k_look_ahead=5): 
        super().__init__(dataset, window_length, mode=mode)
        self.k_look_ahead = k_look_ahead
    
    def train(self): 
        self.lookahead_dict = {}
        for partition in self.dataset:
            for seq in partition: 
                for i in range(len(seq) - self.k_look_ahead):
                    pair = (seq[i], seq[i + self.k_look_ahead])
                    self.lookahead_dict[pair] = self.lookahead_dict.get(pair, 0) + 1
        return self.lookahead_dict

    def predict(self, test_dataset): 
        test_dataset = self.partition(test_dataset)

        anomaly_score = []
        for partition in test_dataset: 
            for sequence in partition:
                anomalies = []
                for i in range(len(sequence) - self.k_look_ahead):
                    pair = (sequence[i], sequence[i + self.k_look_ahead])
                    anomalies.append(self.lookahead_dict.get(pair, 0))

            anomaly_score.append()

        return anomaly_score

## Unsupervised SVM

In [ ]:
   
class UnsupervisedSVM(WindowBasedStruct): 
    def __init__(self, dataset, window_length, mode="average"):
        super().__init__(dataset, window_length, mode=mode)
        self.dataset = self.one_hot_encoding(dataset)

    def one_hot_encoding(self): 
        new_dataset = []
        for partition in self.dataset:
            one_hot_encoded = []
            for seq in partition:
                mat = np.zeros((self.n_symbols, len(seq)), dtype=int)

                for pos, symbol in enumerate(seq):
                    s_idx = self.symbol_to_idx[symbol]
                    mat[s_idx, pos] = 1

                one_hot_encoded.append(mat)
            new_dataset.append(one_hot_encoded)
        return new_dataset
    
    def train(self): 
        self.svm = OneClassSVM(gamma='auto').fit(self.dataset)
        return self.svm
    
    def predict(self, test): 
        test = self.partition(test)
        test = self.one_hot_encoding(test)
        return self.svm.predict(test) 

## Normal Dictionary method

In [ ]:
class NormalDictionary(WindowBasedStruct): 
    def __init__(self, dataset, window_length, mode="average"): 
        super().__init__(dataset, window_length, mode=mode)

    def train(self): 
        self.frequency_dictionary = {}
        for partition in self.dataset: 
            for sequence in partition: 
                sequence = tuple(sequence)
                if sequence in self.frequency_dictionary.keys():
                    self.frequency_dictionary[tuple(sequence)] = self.frequency_dictionary.get(sequence) + 1

        return self.frequency_dictionary

    def predict(self, test_dataset):
        test_dataset = self.partition(test_dataset) 

        anomaly_scores = []
        for partition in test_dataset: 
            anomalies = []
            for sequence in partition: 
                anomalies.append(self.frequency_dictionary.get(tuple(sequence)))
            
            score = self.process_anomaly(anomalies)
            anomaly_scores.append(score)

        return anomaly_scores

## T-side Algorithm 

In [ ]:
class TSIDE(WindowBasedStruct): 
    def __init__(self, dataset, window_length, mode="average"): 
        super().__init__(dataset, window_length, mode=mode)

# IV) Hidden Markov Models :

In [89]:
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
class ViterbiTrainerHMM:
    def __init__(self, n_states, n_symbols, max_iter=50, tol=1e-3):
        self.n_states = n_states
        self.n_symbols = n_symbols
        self.max_iter = max_iter
        self.tol = tol

        A = np.random.rand(n_states, n_states)
        self.A = A / A.sum(axis=1, keepdims=True)

        B = np.random.rand(n_states, n_symbols)
        self.B = B / B.sum(axis=1, keepdims=True)

        self.pi = np.full(n_states, 1.0/n_states)
        # to map the alphabets into numbers
        self.encoder = LabelEncoder()
        self.encoder.fit(list(ALPHABET[:n_symbols]))

    def _encode(self, sequence):
        return self.encoder.transform(sequence)
    
    def _viterbi(self, O):
        """
        Viterbi algorithm for a single sequence O:
        O: array of shape (T,) containing integer-coded observations
        returns:
          states: most likely state sequence
          prob: probability of that sequence under the model
        """
        T = len(O)
        delta = np.zeros((T, self.n_states))
        psi = np.zeros((T, self.n_states), dtype=int)
        O = self._encode(O)
        # Initialization
        delta[0, :] = self.pi * self.B[:, O[0]]
        psi[0, :] = 0

        # Recursion
        for t in range(1, T):
            for j in range(self.n_states):
                vals = delta[t-1, :] * self.A[:, j]
                i_star = np.argmax(vals)
                delta[t, j] = vals[i_star] * self.B[j, O[t]]
                psi[t, j] = i_star

        # Termination
        p_star = np.max(delta[T-1, :])
        q_star = np.zeros(T, dtype=int)
        q_star[T-1] = np.argmax(delta[T-1, :])

        # Backtrack
        for t in range(T-2, -1, -1):
            q_star[t] = psi[t+1, q_star[t+1]]

        return q_star, p_star

    def fit(self, dataset):
        """
        Fit the HMM parameters to the dataset using Viterbi training.
        dataset: list of sequences, each sequence is a list or array of integer-coded observations.
        """
        prev_log_likelihood = -np.inf
        for iteration in range(self.max_iter):
            # Accumulators for counts
            pi_counts = np.zeros(self.n_states)
            A_counts = np.zeros((self.n_states, self.n_states))
            B_counts = np.zeros((self.n_states, self.n_symbols))

            total_log_likelihood = 0.0

            # For each sequence, run Viterbi to get most likely path
            for O in dataset:
                states, seq_prob = self._viterbi(O)
                O = self._encode(O)
                total_log_likelihood += np.log(seq_prob + 1e-12)

                # Update counts based on the Viterbi path
                pi_counts[states[0]] += 1
                for t in range(len(O)):
                    B_counts[states[t], O[t]] += 1
                    if t > 0:
                        A_counts[states[t-1], states[t]] += 1

            # Normalize to get new parameters
            self.pi = pi_counts / pi_counts.sum() if pi_counts.sum() > 0 else self.pi

            row_sums_A = A_counts.sum(axis=1, keepdims=True)
            row_sums_A[row_sums_A == 0] = 1e-12
            self.A = A_counts / row_sums_A

            row_sums_B = B_counts.sum(axis=1, keepdims=True)
            row_sums_B[row_sums_B == 0] = 1e-12
            self.B = B_counts / row_sums_B

            # Check for convergence
            if np.abs(total_log_likelihood - prev_log_likelihood) < self.tol:
                break
            prev_log_likelihood = total_log_likelihood

    def predict(self, dataset):
        """
        Predict the most likely state sequences for a batch of sequences.
        """
        predictions = []
        for O in dataset:
            states, _ = self._viterbi(O)
            predictions.append(states)
        return predictions

    def score(self, dataset):
        """
        Compute the total log-likelihood of the dataset under the current model parameters using Viterbi best paths.
        This is not the full likelihood (like forward algorithm gives), but the likelihood of the Viterbi path.
        """
        total_log_likelihood = 0.0
        for O in dataset:
            _, seq_prob = self._viterbi(O)
            total_log_likelihood += np.log(seq_prob + 1e-12)
        return total_log_likelihood


In [90]:
class BaumWelchHMM: 
    def __init__(self, n_symbols, n_states, max_iter=100, tolerance=1e-3):
        self.n_symbols = n_symbols
        self.n_states = n_states
        self.symbols = {elt:i for i, elt in enumerate(ALPHABET[:n_symbols])}

        self.tolerance = tolerance 
        self.max_iter = max_iter

        A = np.random.rand(n_states, n_states)
        self.transition_matrix = A / A.sum(axis=1, keepdims=True)

        B = np.random.rand(n_states, n_symbols)
        self.emission_matrix  = B / B.sum(axis=1, keepdims=True)

        self.initial_state = np.full(self.n_states, 1/self.n_states)

    def init_fraction(self): 
        self.gamma_first = np.zeros(self.n_states)
        self.transition_numerator = np.zeros((self.n_states, self.n_states))
        self.emission_numerator = np.zeros((self.n_states, self.n_symbols))    
        self.transition_denominator = np.zeros((self.n_states))  
        self.emission_denominator = np.zeros((self.n_states))  

    def one_hot_encode(self, sequence):
        one_hot_encoded = np.zeros((len(sequence), self.n_symbols), dtype=int)
        one_hot_encoded[np.arange(len(sequence)), sequence] = 1
        return one_hot_encoded

    def forward_scaled(self, sequence):
        T = len(sequence)
        alpha = np.zeros((T, self.n_states))
        c = np.zeros(T)  # scaling factors

        # Initialization
        for i in range(self.n_states):
            alpha[0, i] = self.initial_state[i] * self.emission_matrix[i, sequence[0]]
        c[0] = 1.0 / (np.sum(alpha[0, :]) + 1e-300)
        alpha[0, :] *= c[0]

        # Recursion
        for t in range(1, T):
            for j in range(self.n_states):
                alpha[t, j] = np.sum(alpha[t-1, :] * self.transition_matrix[:, j]) * self.emission_matrix[j, sequence[t]]
            c[t] = 1.0 / (np.sum(alpha[t, :]) + 1e-300)
            alpha[t, :] *= c[t]

        return alpha, c

    def backward_scaled(self, sequence, c):
        T = len(sequence)
        beta = np.zeros((T, self.n_states))
        beta[T-1, :] = 1.0 * c[T-1]  

        for t in range(T-2, -1, -1):
            for i in range(self.n_states):
                beta[t, i] = np.sum(self.transition_matrix[i, :] * self.emission_matrix[:, sequence[t+1]] * beta[t+1, :])
            beta[t, :] *= c[t]

        return beta

    
    def convert_dataset(self, dataset): 
        new_dataset = []
        for sequence in dataset: 
            number_sequence = [self.symbols[letter] for letter in sequence]
            new_dataset.append(number_sequence)
        return new_dataset
    
    def E_step(self, sequence):
        n = len(sequence)
        alpha, c = self.forward_scaled(sequence)
        beta = self.backward_scaled(sequence, c)

        log_prob = -np.sum(np.log(c + 1e-300))
        probability = np.exp(log_prob)

        gamma = (alpha * beta)  
        gamma = gamma / (np.sum(gamma, axis=1, keepdims=True) + 1e-300)

        xi = np.zeros((n-1, self.n_states, self.n_states))
        for t in range(n-1):
            denom = np.sum(alpha[t, :] * beta[t, :]) + 1e-300
            for i in range(self.n_states):
                xi[t, i, :] = (alpha[t, i] * self.transition_matrix[i, :] *
                            self.emission_matrix[:, sequence[t+1]] * beta[t+1, :]) / denom

        return gamma, xi, probability


    def M_step(self): 
        # print(self.gamma_first)
        # print(self.transition_numerator, self.transition_denominator)
        # print(self.emission_numerator, self.emission_denominator)
        # print("=====================================")
        self.initial_state = self.gamma_first
        self.transition_matrix = self.transition_numerator / (self.transition_denominator[:, np.newaxis] + 1e-12)
        self.emission_matrix = self.emission_numerator / (self.emission_denominator[:, np.newaxis] + 1e-12)

        self.transition_matrix /= self.transition_matrix.sum(axis=1)
        self.emission_matrix /= self.emission_matrix.sum(axis=1)[:, np.newaxis]

    
    def Baum_Welch(self, dataset):
        n_sequence = len(dataset)
        dataset = self.convert_dataset(dataset)
        self.init_fraction()  
        
        log_likelihood = 0.0
        
        for sequence in dataset:
            mask = self.one_hot_encode(sequence)

            gamma, xi, probability = self.E_step(sequence)

            self.gamma_first += gamma[0, :] / n_sequence

            self.transition_numerator += np.sum(xi[:-1], axis=0)
            self.emission_numerator += gamma.T @ mask
            self.transition_denominator += np.sum(gamma[:-1], axis=0) 
            self.emission_denominator += np.sum(gamma, axis=0)

            log_likelihood += np.log(probability + 1e-12)
        
        self.M_step()

        return log_likelihood

    def fit(self, dataset):
        prev_log_likelihood = -np.inf
        log_likelihood = 0 

        with tqdm(total=self.max_iter, desc="EM Algorithm Progress", unit="step") as pbar:
            for i in range(self.max_iter):
                prev_log_likelihood = log_likelihood  
                log_likelihood = self.Baum_Welch(dataset)
                pbar.set_postfix({"Log-Likelihood": log_likelihood})
                pbar.update(1)
                

    def predict_sample(self, sequence): 
        alpha = self.forward(sequence)
        probability = np.sum(alpha[-1, :])
        return -np.log(probability + 1e-12)
    
    def predict(self, dataset): 
        dataset = self.convert_dataset(dataset)
        scores = []
        for sequence in dataset:
            scores.append(self.predict_sample(sequence))
        return np.array(scores)

In [91]:
import numpy as np
from tqdm import tqdm

class ImprovedHiddenMarkovModel:
    def __init__(self, n_symbols, n_states, max_iter=100, tolerance=1e-4, 
                 smoothing=1e-10, random_seed=42):
        """
        Improved HMM implementation with numerical stability
        
        Parameters:
        - n_symbols: Number of possible observation symbols
        - n_states: Number of hidden states
        - max_iter: Maximum number of EM iterations
        - tolerance: Convergence threshold for log-likelihood
        - smoothing: Small value to prevent zero probabilities
        - random_seed: Seed for reproducibility
        """
        np.random.seed(random_seed)
        self.n_symbols = n_symbols
        self.n_states = n_states
        self.max_iter = max_iter
        self.tolerance = tolerance
        self.smoothing = smoothing
        
        # Initialize parameters with small smoothing
        self.transition_matrix = self._initialize_transition_matrix()
        self.emission_matrix = self._initialize_emission_matrix()
        self.initial_state = self._initialize_initial_state()

    def _initialize_transition_matrix(self):
        """Initialize transition matrix with smoothing"""
        A = np.random.random((self.n_states, self.n_states)) + self.smoothing
        return A / A.sum(axis=1, keepdims=True)

    def _initialize_emission_matrix(self):
        """Initialize emission matrix with smoothing"""
        B = np.random.random((self.n_states, self.n_symbols)) + self.smoothing
        return B / B.sum(axis=1, keepdims=True)

    def _initialize_initial_state(self):
        """Initialize initial state probabilities"""
        pi = np.random.random(self.n_states) + self.smoothing
        return pi / pi.sum()

    def _log_forward_algorithm(self, sequence):
        """
        Perform forward algorithm in log space to prevent underflow
        
        Returns:
        - log_alpha: Log-space forward probabilities
        - log_prob: Log probability of the sequence
        """
        log_initial = np.log(self.initial_state + self.smoothing)
        log_transition = np.log(self.transition_matrix + self.smoothing)
        log_emission = np.log(self.emission_matrix + self.smoothing)

        log_alpha = np.zeros((len(sequence), self.n_states))
        log_alpha[0] = log_initial + log_emission[:, sequence[0]]
        
        for t in range(1, len(sequence)):
            for j in range(self.n_states):
                log_alpha[t, j] = logsumexp(
                    log_alpha[t-1] + log_transition[:, j]
                ) + log_emission[j, sequence[t]]
        
        log_prob = logsumexp(log_alpha[-1])
        return log_alpha, log_prob

    def _log_backward_algorithm(self, sequence, log_prob):
        """
        Perform backward algorithm in log space
        
        Returns:
        Log-space backward probabilities
        """
        log_transition = np.log(self.transition_matrix + self.smoothing)
        log_emission = np.log(self.emission_matrix + self.smoothing)

        log_beta = np.zeros((len(sequence), self.n_states))
        log_beta[-1] = 0.0

        for t in range(len(sequence)-2, -1, -1):
            for i in range(self.n_states):
                log_beta[t, i] = logsumexp(
                    log_transition[i, :] + 
                    log_emission[:, sequence[t+1]] + 
                    log_beta[t+1]
                )
        
        return log_beta

    def _compute_gamma_xi(self, sequence, log_alpha, log_beta, log_prob):
        """
        Compute posterior probabilities gamma and xi
        """
        log_transition = np.log(self.transition_matrix + self.smoothing)
        log_emission = np.log(self.emission_matrix + self.smoothing)

        # Compute log gamma (state posteriors)
        log_gamma = log_alpha + log_beta - log_prob
        gamma = np.exp(log_gamma)

        # Compute log xi (transition posteriors)
        xi = np.zeros((len(sequence)-1, self.n_states, self.n_states))
        for t in range(len(sequence)-1):
            for i in range(self.n_states):
                for j in range(self.n_states):
                    xi[t, i, j] = np.exp(
                        log_alpha[t, i] + 
                        log_transition[i, j] + 
                        log_emission[j, sequence[t+1]] + 
                        log_beta[t+1, j] - 
                        log_prob
                    )
            xi[t] /= xi[t].sum()

        return gamma, xi

    def fit(self, sequences):
        """
        EM algorithm for parameter estimation
        
        Args:
        sequences: List of sequences, where each sequence is a list of symbol indices
        """
        prev_log_likelihood = -np.inf

        for iteration in tqdm(range(self.max_iter), desc="EM Iterations"):
            # Accumulators for expected counts
            total_gamma_0 = np.zeros(self.n_states)
            total_transition = np.zeros_like(self.transition_matrix)
            total_emission = np.zeros_like(self.emission_matrix)
            log_likelihood = 0.0

            # E-step
            for sequence in sequences:
                # Compute log-space forward and backward probabilities
                log_alpha, log_prob = self._log_forward_algorithm(sequence)
                log_beta = self._log_backward_algorithm(sequence, log_prob)
                
                # Compute posterior probabilities
                gamma, xi = self._compute_gamma_xi(sequence, log_alpha, log_beta, log_prob)
                
                # Accumulate statistics
                total_gamma_0 += gamma[0]
                total_transition += xi.sum(axis=0)
                
                for t, sym in enumerate(sequence):
                    total_emission[:, sym] += gamma[t]
                
                log_likelihood += log_prob

            # M-step: Update parameters with smoothing
            self.initial_state = total_gamma_0 / total_gamma_0.sum()
            self.transition_matrix = (total_transition + self.smoothing) 
            self.transition_matrix /= self.transition_matrix.sum(axis=1, keepdims=True)
            
            self.emission_matrix = (total_emission + self.smoothing)
            self.emission_matrix /= self.emission_matrix.sum(axis=1, keepdims=True)

            # Check convergence
            if np.abs(log_likelihood - prev_log_likelihood) < self.tolerance:
                break
            
            prev_log_likelihood = log_likelihood

        return self

def logsumexp(x):
    """
    Numerically stable log-sum-exp trick
    """
    max_x = np.max(x)
    return max_x + np.log(np.sum(np.exp(x - max_x)))

In [92]:
def generate_random_probability_matrix(n_rows, n_cols):
    matrix = np.random.rand(n_rows, n_cols)  
    row_sums = matrix.sum(axis=1, keepdims=True)
    matrix = matrix / row_sums
    return matrix

In [93]:
n_symbols = 4
n_states = 3

transition_matrix1 = np.array([
    [0.7, 0.2, 0.1],
    [0.1, 0.7, 0.2],
    [0.2, 0.1, 0.7], 
])

emission_matrix1 = np.array([
    [0.5, 0.2, 0.1, 0.2],
    [0.1, 0.6, 0.1, 0.2],
    [0.2, 0.2, 0.4, 0.2]
])


transition_matrix2 = generate_random_probability_matrix(n_states, n_states)
emission_matrix2 = generate_random_probability_matrix(n_states, n_symbols)

generator = MarkovSequenceGenerator(transition_matrix=transition_matrix1, 
                               emission_matrix=emission_matrix1, 
                               sequence_length=10)
train_dataset = generator.generate_all_sequences(initial_state=0, n_sequence=50)


dataset_generator = MarkovDatasetGenerator(transition_matrices=[transition_matrix1, transition_matrix2], 
                                                 emission_matrices=[emission_matrix1, emission_matrix2], 
                                                 sequence_length=10,)

test_dataset, labels = dataset_generator.generate()

In [94]:
train_dataset[0]

['A', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'D', 'C']

In [95]:
hmm = ViterbiTrainerHMM(n_symbols=n_symbols, n_states=n_states, max_iter=100, tol=1e-3)

In [96]:
[i for i in range(len(hmm._encode(train_dataset[0])))]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [97]:
hmm.fit(train_dataset[:100])

# train_dataset = hmm.convert_dataset(train_dataset)
# cat = CategoricalHMM(n_components=3, algorithm='viterbi', n_iter=10)
# cat.fit(train_dataset)

In [104]:
claude_dataset = BaumWelchHMM(n_symbols, n_states=n_states, max_iter=10, tolerance=1e-3).convert_dataset(train_dataset)

In [98]:
# test_dataset = hmm.convert_dataset(test_dataset)
hmm.emission_matrix

AttributeError: 'ViterbiTrainerHMM' object has no attribute 'emission_matrix'

In [ ]:
prediction = cat.predict(test_dataset)
prediction

array([2, 0, 0, ..., 2, 2, 2])

In [ ]:
cat.predict_proba(test_dataset)

array([[5.73355959e-04, 5.74615691e-10, 9.99426643e-01],
       [8.77011591e-01, 1.12507443e-01, 1.04809666e-02],
       [7.73528788e-01, 2.09231999e-01, 1.72392127e-02],
       ...,
       [6.56708989e-01, 7.20222327e-02, 2.71268779e-01],
       [6.56258090e-01, 1.87903224e-01, 1.55838686e-01],
       [5.18080705e-01, 2.36853056e-01, 2.45066239e-01]])

In [101]:
predictions = hmm.score(test_dataset)
predictions

np.float64(-2900.2197740127285)